# CS-660 Semester Project Notebook 1

## Stan Rosenbaum

Fall 2017

Anaconda 5 / Python 3

Using Keras with TensorFlow as back end.


First we init stuff.

In [1]:
# Load the Basic Python Libraries
import os
import csv
import PIL
import pickle
import random
import datetime

# Load my Data Management Module
import CS660DataManagement as csDM

# load numpy
import numpy as np

# Load Keras Studd
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import Model, Sequential
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from keras.initializers import glorot_uniform

from keras.callbacks import TensorBoard

from keras.utils.np_utils import to_categorical



# Other
import pydot
from IPython.display import SVG


# Not sure what this is:
# from kt_utils import *

import keras.backend as K
K.set_image_data_format('channels_last')
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

%matplotlib inline

# Get Processed Data Directory.
processedDataDir = os.path.join( os.getcwd(), os.pardir, 'DATA', 'PROCESSED' )
combinedDataDir = os.path.join( os.getcwd(), os.pardir, 'DATA', 'COMBINED' )
pickleDataDir = os.path.join( os.getcwd(), os.pardir, 'DATA', 'PICKLES' )

testImageColorFile = os.path.join( os.getcwd(), os.pardir, 'DATA', 'COMBINED', 'ICOLOR', 'TEST.png' )
testImageDepthFile = os.path.join( os.getcwd(), os.pardir, 'DATA', 'COMBINED', 'IDEPTH', 'TEST.png' )
testCSVFile = os.path.join( os.getcwd(), os.pardir, 'DATA', 'COMBINED', 'PCLOUD', 'TEST.csv' )

combinedDataDir = os.path.join( os.getcwd(), os.pardir, 'DATA', 'COMBINED' )

imageDirs = ['ICOLOR', 'IDEPTH']
csvDirs = ['PCLOUD']

allDataFlavors = imageDirs + csvDirs

#Load main data file.
searCSVInfoFile = os.path.join( combinedDataDir, 'SEAR_DC_INFO.csv' )

csDM.CS660DataManagementCheck()

Using TensorFlow backend.


CS660DataManagementCheck Imported


## Generate the Learning and Verification sets

In [2]:
print( len( csDM.getListOfDataCSVFileKeys() ) )
csDM.reportStats()
#csDM.createLearningAndVerificationPickles()


901
Number of Everything
UPs: 278
DOWNs: 201
NAs: 204
HOLEs: 218


### Load a Training Set.
#### (Basing this all on the Coursera Course Code)



In [3]:
def reshape(trainingSet):
    """
    This is to add the 'Channels' Dimension for Keras.
    """
    getShape = trainingSet.shape
    newShape = getShape + (1,)
    return trainingSet.reshape(newShape)
    

X_train, Y_train, X_test, Y_test = csDM.load_dataset( '1', 'ICOLOR' )



print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))

X_train = reshape(X_train)
X_test = reshape(X_test)

y_train_binary = to_categorical(Y_train, 4)
y_test_binary = to_categorical(Y_test, 4)

print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("y_train_binary shape: " + str(y_train_binary.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))
print ("y_test_binary shape: " + str(y_test_binary.shape))



number of training examples = 100
number of test examples = 20
X_train shape: (100, 480, 640, 1)
Y_train shape: (100, 1)
y_train_binary shape: (100, 4)
X_test shape: (20, 480, 640, 1)
Y_test shape: (20, 1)
y_test_binary shape: (20, 4)


## 2 - Building a model in Keras



In [58]:
csDM.CS660DataManagementCheck()

CS660DataManagementCheck Imported


In [23]:
import keras
from quiver_engine import server

# input image dimensions
img_rows, img_cols = 480, 640

num_classes = 4

x_train, y_train, x_test, y_test = csDM.load_dataset( '1', 'ICOLOR' )

x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train_as_category = to_categorical(y_train, num_classes)
y_test_as_category = to_categorical(y_test, num_classes)


x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
print('x_train shape:', x_train.shape)
print(y_train_as_category.shape, 'y_train shape')
print(y_test_as_category.shape, 'y_test shape')



model = Sequential()
model.add(
    Conv2D(5, 
           kernel_size=(5, 5),
           activation='relu',
           input_shape=input_shape
          )
)
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(
    Conv2D(
        10, 
        kernel_size=(2, 2), 
        activation='relu')
)
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Flatten())

model.add(Dense(512, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.25))

model.add(Dense(512, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.25))

# model.add(Dense(512, activation='relu'))
# model.add(Dropout(0.25))
# model.add(Dense(512, activation='relu'))
# model.add(Dropout(0.25))
# model.add(Dense(512, activation='relu'))
# model.add(Dropout(0.25))
# model.add(Dense(512, activation='relu'))
# model.add(Dropout(0.25))

# model.add(Dense(512, activation='relu'))
# model.add(Dropout(0.25))
# model.add(Dense(512, activation='relu'))
# model.add(Dropout(0.25))
# model.add(Dense(512, activation='relu'))
# model.add(Dropout(0.25))
# model.add(Dense(512, activation='relu'))
# model.add(Dropout(0.25))

model.add(Dense(num_classes, activation='softmax'))

model.compile(
    loss=keras.losses.categorical_crossentropy,
    optimizer=keras.optimizers.Adam(),
    metrics=['categorical_accuracy']
)

# server.launch(model)

TBoardCallback = keras.callbacks.TensorBoard(
    log_dir='../TENSOR_LOGS/run_test', 
    histogram_freq=0,
    write_graph=True,
    write_images=True
)

model.fit(x_train,
          y_train_as_category,
          batch_size=16,
          epochs=32,
          verbose=1,
          validation_data=(x_test, y_test_as_category),
          callbacks = [TBoardCallback]
         )



x_train shape: (100, 480, 640, 1)
100 train samples
20 test samples
x_train shape: (100, 480, 640, 1)
(100, 4) y_train shape
(20, 4) y_test shape
Train on 100 samples, validate on 20 samples
Epoch 1/32
100/100 [==============================] - 16s - loss: 1.8628 - categorical_accuracy: 0.3000 - val_loss: 1.4357 - val_categorical_accuracy: 0.2500
Epoch 2/32
100/100 [==============================] - 16s - loss: 1.7874 - categorical_accuracy: 0.2300 - val_loss: 1.4272 - val_categorical_accuracy: 0.2500
Epoch 3/32
100/100 [==============================] - 15s - loss: 1.5364 - categorical_accuracy: 0.2900 - val_loss: 1.3583 - val_categorical_accuracy: 0.2500
Epoch 4/32
100/100 [==============================] - 15s - loss: 1.3883 - categorical_accuracy: 0.2600 - val_loss: 1.3391 - val_categorical_accuracy: 0.2500
Epoch 5/32
100/100 [==============================] - 15s - loss: 1.3426 - categorical_accuracy: 0.2500 - val_loss: 1.3009 - val_categorical_accuracy: 0.3000
Epoch 6/32
100/100 

In [24]:
score = model.evaluate(x_test, y_test_as_category, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

# serialize model to JSON
model_json = model.to_json()
with open("../MODELS/modelTEST.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("../WEIGHTS/modelTEST.h5")
print("Saved model to disk")

Test loss: 1.56756234169
Test accuracy: 0.699999988079
Saved model to disk


In [25]:
# Predict
print(x_test.shape)
results = model.predict_classes(x_test, verbose=1)
print(results)
for index in range(len(x_test)):
    print('prediction of: ', results[index], " VS ", y_test[index])



(20, 480, 640, 1)
20/20 [==============================] - 1s
[2 2 1 3 1 2 1 0 1 0 2 3 3 3 3 3 1 0 1 0]
prediction of:  2  VS  [ 2.]
prediction of:  2  VS  [ 2.]
prediction of:  1  VS  [ 1.]
prediction of:  3  VS  [ 2.]
prediction of:  1  VS  [ 1.]
prediction of:  2  VS  [ 1.]
prediction of:  1  VS  [ 2.]
prediction of:  0  VS  [ 0.]
prediction of:  1  VS  [ 1.]
prediction of:  0  VS  [ 1.]
prediction of:  2  VS  [ 2.]
prediction of:  3  VS  [ 3.]
prediction of:  3  VS  [ 3.]
prediction of:  3  VS  [ 3.]
prediction of:  3  VS  [ 3.]
prediction of:  3  VS  [ 3.]
prediction of:  1  VS  [ 0.]
prediction of:  0  VS  [ 0.]
prediction of:  1  VS  [ 0.]
prediction of:  0  VS  [ 0.]


In [5]:
os.getcwd()

'/Users/staque/Development/GitHub/CS-660-Semester-Project/ANACONDA'

Saved model to disk
